## Maestría en Inteligencia Artificial Aplicada (MNA)
### Proyecto Integrador
### Dra. Grettel Barceló Alonso / Dr. Carlos Alberto Villaseñor Padilla
### Avance 3. Baseline

### Integrantes
- A01794457 - Iossif Moises Palli Laura
- A01793984 - Brenda Zurazy Rodríguez Pérez
- A01794630 - Jesús Ramseths Echeverría Rivera

In [ ]:
!pip install datasets peft bitsandbytes
!pip install -U bitsandbytes

In [ ]:
# Paqueterías a utilizar
import pandas as pd
from datasets import load_dataset
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer
from transformers import LlamaTokenizer, LlamaForCausalLM
from peft import LoraConfig, get_peft_model
from transformers import BitsAndBytesConfig
from torch.utils.data import DataLoader
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu

Primero se hace la **configuración para la cuantización** del modelo LLM utilizando la función BitsAndBytes.

La cuantización es una técnica que reduce el tamaño de los modelos y mejora la eficiencia de la inferencia, permitiendo que los modelos se ejecuten más rápidamente y con menos memoria, sin una pérdida significativa en la calidad.

In [ ]:
# Cuantización del modelo
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

Se inicia sesión en el Hugging Face Hub utilizando un token de autenticación.

In [ ]:
# Inicio de sesión en el Hub de Hugging Face
from huggingface_hub import login

# Token de huggingface
login('')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Se carga el modelo de lenguaje preentrenado de Hugging Face, específicamente **el modelo LLaMA 3.2**, junto con su tokenizador.

Llama 3.2, es un modelo que incluye LLM de visión (11B y 90B) y de texto (1B y 3B) para dispositivos móviles y de gama alta, optimizados para tareas como recuperación de conocimientos y seguimiento de instrucciones, con soporte para hardware Qualcomm y MediaTek.

**Bibliografía:**

Meta. (2024, 7 octubre). Newsroom | Meta. Meta. https://about.fb.com/news/

In [ ]:
# Carga de modelo Llama
MODEL_NAME = 'meta-llama/Llama-3.2-3B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, quantization_config=quantization_config, device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Posteriormente se configura y aplica **LoRA (Low-Rank Adaptation)** al modelo.

LoRA es un método que acelera el entrenamiento de modelos grandes mientras consume menos memoria, ya que en lugar de ajustar todos los millones (o incluso billones) de parámetros de un modelo, LoRA se enfoca solo en modificar una pequeña parte de ellos, ahorrando recursos computacionales y tiempo.

**Bibliografía:**

Low-Rank Adaptation of Large Language Models (LoRA). (s. f.). https://huggingface.co/docs/diffusers/v0.21.0/training/lora#lowrank-adaptation-of-large-language-models-lora


In [ ]:
# Configuración de LoRA (Low-Rank Adaptation)
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Módulos a los que se aplicará LoRA
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

Se hace la carga de los datos de estructura de preguntas y repuestas.

In [ ]:
# Carga de datos
dataset = load_dataset('csv', data_files='q_a_db.csv')

Se dividen los datos en entrenamiento y prueba.

In [ ]:
split_dataset = dataset['train'].train_test_split(test_size=0.2)

# Asignación de la partición
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [ ]:
print('Tamaño de entrenamiento:', train_dataset.shape[0])
print('Tamaño de prueba:', test_dataset.shape[0])

Tamaño de entrenamiento: 914
Tamaño de prueba: 229


Se contruye una función de tokenización llamada **tokenize_function** que nos servirá para conviertir el texto en secuencias de tokens que el modelo puede procesar.

La función concatena cada pregunta y respuesta de la base de datos, después generará la tokenización y se etiquetarán cada una de ellas, posteriormente, medirá la longitud de los tokens correspondientes al prompt, esto se hace para diferenciar los tokens que corresponden a la pregunta y respuesta y finalmente lo que hará la función es enmascarar los tokens correspondientes al prompt.

In [ ]:
# Se define una función de tokenización llamada tokenize_function
def tokenize_function(example):
    # Concatenar prompt y respuesta
    full_text = example['question'] + example['answer']

    # Tokenizar
    tokenized_example = tokenizer(
        full_text,
        truncation=True,
        padding='max_length',
        max_length=500,)

    # Crear etiquetas
    labels = tokenized_example['input_ids'].copy()

    # Calcular la longitud del prompt
    prompt_length = len(tokenizer(
        example['question'],
        add_special_tokens=False
    )['input_ids'])

    # Enmascarar los tokens del prompt
    labels[:prompt_length] = [-100] * prompt_length

    tokenized_example['labels'] = labels
    return tokenized_example


train_tokenized_dataset = train_dataset.map(tokenize_function, batched=False)
test_tokenized_dataset = test_dataset.map(tokenize_function, batched=False)


Map:   0%|          | 0/914 [00:00<?, ? examples/s]

Map:   0%|          | 0/229 [00:00<?, ? examples/s]

Una vez que se transforme la base de datos original en un nuevo dataset donde cada registro está tokenizado y por lo tanto listo para el entrenamiento de un modelo.

In [ ]:
training_args = TrainingArguments(
    output_dir='./resultado_lora',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=16,
    num_train_epochs=5,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
)

In [ ]:
# Entrenamiento del modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
10,3.232700
20,0.117000
30,0.093000
40,0.088000
50,0.083600
60,0.081400
70,0.080800


TrainOutput(global_step=70, training_loss=0.5395010854516711, metrics={'train_runtime': 1229.7151, 'train_samples_per_second': 3.716, 'train_steps_per_second': 0.057, 'total_flos': 3.7847088095232e+16, 'train_loss': 0.5395010854516711, 'epoch': 4.890829694323144})

La tabla anterior nos muestra cómo disminuye la pérdida durante el entrenamiento del modelo, vemos que a partir del paso 20 y hasta el final, la pérdida es bastante baja (llegando a 0.080800 en el paso 70 y fluctuando ligeramente).

Esto indica que el **modelo aprendió correctamente** y se está ajustando bien a los datos, logrando una buena mejora con el tiempo.

In [ ]:
# Se guarda el modelos entrenado y la tokenización
model.save_pretrained('llama-3.2-3b-fine-tuning')
tokenizer.save_pretrained('llama-3.2-3b-fine-tuning')

('llama-3.2-3b-fine-tuning/tokenizer_config.json',
 'llama-3.2-3b-fine-tuning/special_tokens_map.json',
 'llama-3.2-3b-fine-tuning/tokenizer.json')

### Evaluación

In [ ]:
#  Importa las librerías para trabajar con modelos ajustados mediante PEFT (Parameter-Efficient Fine-Tuning) y la librería de PyTorch.
from peft import PeftModel
import torch

Se carga el modelo LLaMA preentrenado, previamente se aplicarán técnicas de cuantización para reducir el uso de recursos para que después poder hacer inferencia.

In [ ]:
model_name = './llama-3.2-3b-fine-tuning/' #'meta-llama/Llama-3.2-3B-Instruct' #

# Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Configuración de cuantización
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

# Cargar el modelo con cuantización
model = LlamaForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map='auto'
)

# # Carga
# model = PeftModel.from_pretrained(model, model_name)

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=3072, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=3072, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear8bitLt(in_features=3072, out_features=1024, bias=False)
          (v_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_

Probando el modelo

In [ ]:
# Prompt de entrada
prompt = """¿Qué necesito para un sistema de pagos en México?"""

# Tokenizar
inputs = tokenizer(prompt, return_tensors='pt')

# Mover los tensores
inputs = {key: value.to(model.device) for key, value in inputs.items()}

# Generar la respuesta
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=1000,    # Número máximo de tokens a generar
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2
    )

# Decodificar
respuesta = tokenizer.decode(output[0], skip_special_tokens=True)

# Extraer
respuesta_generada = respuesta[len(prompt):].strip()

print("Respuesta del modelo:")
print(respuesta_generada)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Respuesta del modelo:
Licencia del Banco y cumplimiento con las regulaciones financieras.


### Métricas

In [ ]:
# Se seleccionan dos muestras debido a la capacidad
test_samples = test_dataset.select(range(2))
test_tokenized_samples = test_samples.map(tokenize_function, batched=False)

In [ ]:
# Carga de datos de prueba
test_dataloader = DataLoader(
    test_tokenized_samples,
    batch_size=8,
    shuffle=False
)

In [ ]:
def generate_responses(model, tokenizer, test_dataset):
    model.eval()
    generated_responses = []
    real_responses = []

    for example in tqdm(test_dataset):
        prompt = example['question']
        real_answer = example['answer']

        # Tokenizar el prompt
        inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

        # Generar la respuesta
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=200,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.2
            )

        # Decodificar la respuesta generada
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        # Extraer solo la respuesta generada (sin el prompt)
        generated_answer = generated_text[len(prompt):].strip()

        generated_responses.append(generated_answer)
        real_responses.append(real_answer)

    return generated_responses, real_responses

In [ ]:
generated_responses, real_responses = generate_responses(model, tokenizer, test_tokenized_samples)

100%|██████████| 2/2 [01:07<00:00, 33.96s/it]


In [ ]:
for i in range(len(generated_responses)):
    reference = [real_responses[i].split()]
    candidate = generated_responses[i].split()
    bleu_score = sentence_bleu(reference, candidate)
    print(f"BLEU Score para la muestra {i+1}: {bleu_score}")

BLEU Score para la muestra 1: 0.033846304491122574
BLEU Score para la muestra 2: 2.6793474497416882e-155


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


# **Conclusiones**

El modelo generó una respuesta en base al prompt, lo procesó a través del modelo LLaMA para generar una respuesta textual y luego mostró la respuesta resultante.

Al utilizar el modelo entrenado para generar respuestas a preguntas específicas, se observan resultados que pueden ser útiles para aplicaciones del mundo real, como sistemas de soporte al cliente o asistentes virtuales.

Es importante mencionar que el modelo al ser Baseline existen posibles mejoras que se pueden hacer a través del ajuste de los valores de los hiperparámetros durante el entrenamiento, el ajuste de estos hiperparámetros causaría una optimización en el rendimiento del modelo.

Por último, también esperamos incorporar un sistema de retroalimentación que permita al modelo aprender de las interacciones reales con los usuarios, esto provocaría mejoras significativas en la calidad de las respuestas y su relevancia en contextos específicos.